In [1]:
!pip3 install http://download.pytorch.org/whl/cu80/torch-0.3.1-cp36-cp36m-linux_x86_64.whl 
!pip3 install torchvision

    100% |████████████████████████████████| 496.4MB 23.2MB/s 
    100% |████████████████████████████████| 51kB 1.7MB/s 
    100% |████████████████████████████████| 5.9MB 225kB/s 


  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


In [2]:
!pip3 install cffi
!pip3 install torchwordemb
!pip3 install torchtext

    100% |████████████████████████████████| 430kB 2.4MB/s 
    100% |████████████████████████████████| 256kB 3.4MB/s 
  Running setup.py bdist_wheel for pycparser ... - \ done
  Stored in directory: /content/.cache/pip/wheels/95/14/9a/5e7b9024459d2a6600aaa64e0ba485325aff7a9ac7489db1b6
Successfully built pycparser
  Running setup.py bdist_wheel for torchwordemb ... - \ | done
  Stored in directory: /content/.cache/pip/wheels/e2/5e/7c/155e2563587ed9b93540a3b9aaf1ffbd8fd3eaee2481751cc1
Successfully built torchwordemb
    100% |████████████████████████████████| 51kB 1.9MB/s 
    100% |████████████████████████████████| 61kB 3.9MB/s 


In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [0]:
import torchwordemb

vocab, vec = torchwordemb.load_word2vec_bin("/path/to/word2vec/model.bin")
print(vec.size())
print(vec[ w2v.vocab["apple"] ] )

In [0]:
embed = nn.Embedding(num_embeddings, embedding_dim)
# pretrained_weight is a numpy matrix of shape (num_embeddings, embedding_dim)
embed.weight.data.copy_(torch.from_numpy(pretrained_weight))

In [0]:
import re

import os

import random

import tarfile

import urllib

from torchtext import data





class TarDataset(data.Dataset):

    """Defines a Dataset loaded from a downloadable tar archive.



    Attributes:

        url: URL where the tar archive can be downloaded.

        filename: Filename of the downloaded tar archive.

        dirname: Name of the top-level directory within the zip archive that

            contains the data files.

    """



    @classmethod

    def download_or_unzip(cls, root):

        path = os.path.join(root, cls.dirname)

        if not os.path.isdir(path):

            tpath = os.path.join(root, cls.filename)

            if not os.path.isfile(tpath):

                print('downloading')

                urllib.request.urlretrieve(cls.url, tpath)

            with tarfile.open(tpath, 'r') as tfile:

                print('extracting')

                tfile.extractall(root)

        return os.path.join(path, '')





class MR(TarDataset):



    url = 'https://www.cs.cornell.edu/people/pabo/movie-review-data/rt-polaritydata.tar.gz'

    filename = 'rt-polaritydata.tar'

    dirname = 'rt-polaritydata'



    @staticmethod

    def sort_key(ex):

        return len(ex.text)



    def __init__(self, text_field, label_field, path=None, examples=None, **kwargs):

        """Create an MR dataset instance given a path and fields.



        Arguments:

            text_field: The field that will be used for text data.

            label_field: The field that will be used for label data.

            path: Path to the data file.

            examples: The examples contain all the data.

            Remaining keyword arguments: Passed to the constructor of

                data.Dataset.

        """

        def clean_str(string):

            """

            Tokenization/string cleaning for all datasets except for SST.

            Original taken from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py

            """

            string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)

            string = re.sub(r"\'s", " \'s", string)

            string = re.sub(r"\'ve", " \'ve", string)

            string = re.sub(r"n\'t", " n\'t", string)

            string = re.sub(r"\'re", " \'re", string)

            string = re.sub(r"\'d", " \'d", string)

            string = re.sub(r"\'ll", " \'ll", string)

            string = re.sub(r",", " , ", string)

            string = re.sub(r"!", " ! ", string)

            string = re.sub(r"\(", " \( ", string)

            string = re.sub(r"\)", " \) ", string)

            string = re.sub(r"\?", " \? ", string)

            string = re.sub(r"\s{2,}", " ", string)

            return string.strip()



        text_field.preprocessing = data.Pipeline(clean_str)

        fields = [('text', text_field), ('label', label_field)]



        if examples is None:

            path = self.dirname if path is None else path

            examples = []

            with open(os.path.join(path, 'rt-polarity.neg'), errors='ignore') as f:

                examples += [

                    data.Example.fromlist([line, 'negative'], fields) for line in f]

            with open(os.path.join(path, 'rt-polarity.pos'), errors='ignore') as f:

                examples += [

                    data.Example.fromlist([line, 'positive'], fields) for line in f]

        super(MR, self).__init__(examples, fields, **kwargs)



    @classmethod

    def splits(cls, text_field, label_field, dev_ratio=.1, shuffle=True, root='.', **kwargs):

        """Create dataset objects for splits of the MR dataset.



        Arguments:

            text_field: The field that will be used for the sentence.

            label_field: The field that will be used for label data.

            dev_ratio: The ratio that will be used to get split validation dataset.

            shuffle: Whether to shuffle the data before split.

            root: The root directory that the dataset's zip archive will be

                expanded into; therefore the directory in whose trees

                subdirectory the data files will be stored.

            train: The filename of the train data. Default: 'train.txt'.

            Remaining keyword arguments: Passed to the splits method of

                Dataset.

        """

        path = cls.download_or_unzip(root)

        examples = cls(text_field, label_field, path=path, **kwargs).examples

        if shuffle: random.shuffle(examples)

        dev_index = -1 * int(dev_ratio*len(examples))



        return (cls(text_field, label_field, examples=examples[:dev_index]),

                cls(text_field, label_field, examples=examples[dev_index:]))

In [0]:
import torch

import torch.nn as nn

import torch.nn.functional as F

from torch.autograd import Variable





class CNN_Text(nn.Module):

    

    def __init__(self, args):

        super(CNN_Text, self).__init__()

        self.args = args

        V = args.embed_num

        D = args.embed_dim

        C = args.class_num

        Ci = 1

        Co = args.kernel_num

        Ks = args.kernel_sizes
        
        G_in=Co
        
        G_out=args.g_out
        
        G_l=args.gru_l
       
        self.embed = nn.Embedding(V, D)

        # self.convs1 = [nn.Conv2d(Ci, Co, (K, D)) for K in Ks]

        self.convs1 = nn.ModuleList([nn.Conv2d(Ci, Co, (K, D)) for K in Ks])

        '''

        self.conv13 = nn.Conv2d(Ci, Co, (3, D))

        self.conv14 = nn.Conv2d(Ci, Co, (4, D))

        self.conv15 = nn.Conv2d(Ci, Co, (5, D))

        '''

        self.dropout = nn.Dropout(args.dropout)

        self.fc1 = nn.Linear(len(Ks)*Co*args.kmax, 256)
        
        #self.fc1 = nn.Linear(len(Ks)*G_out, 256)
        
        #self.fc1 = nn.Linear(len(Ks)*G_out , C)
        
        self.fc2 = nn.Linear(256, C)
        
        #self.rnn=nn.GRU(G_in, G_out, G_l)
        
        
    def kmax_pooling(self, x, dim, k):
      
        index = x.topk(k, dim = dim)[1].sort(dim = dim)[0]
      
        return x.gather(dim, index)
        
       



    def forward(self, x):

        x = self.embed(x)  # (N, W, D)

        if self.args.static:

            x = Variable(x)

        x = x.unsqueeze(1)  # (N, Ci, W, D)

        x = [F.relu(conv(x)).squeeze(3) for conv in self.convs1]  # [(N, Co, W), ...]*len(Ks)
        
        #x = [torch.transpose(i, 2, 0) for i in x]
        
        #x = [torch.transpose(i, 2, 1) for i in x]
    
        #x = [self.rnn(i)[1].squeeze(1) for i in x]



        #x = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in x]  # [(N, Co), ...]*len(Ks)
        
        x = [self.kmax_pooling(i, 2, args.kmax).squeeze(2) for i in x]  # [(N, Co), ...]*len(Ks)
       
        x = torch.cat(x, 1)
        
        x = x.view(x.size(0),-1)



        '''

        x1 = self.conv_and_pool(x,self.conv13) #(N,Co)

        x2 = self.conv_and_pool(x,self.conv14) #(N,Co)

        x3 = self.conv_and_pool(x,self.conv15) #(N,Co)

        x = torch.cat((x1, x2, x3), 1) # (N,len(Ks)*Co)

        '''

        x = self.dropout(x)  # (N, len(Ks)*Co)

        x = self.fc1(x)  # (N, C)
        
        x = F.relu(x)
        
        x = self.dropout(x)  # (N, len(Ks)*Co)

        logit = self.fc2(x)

        return logit
      
      

#    def conv_and_pool(self, x, conv):

 #       x = F.relu(conv(x)).squeeze(3)  # (N, Co, W)

  #      x = F.max_pool1d(x, x.size(2)).squeeze(2)

   #     return x
     

In [0]:
import os

import sys

import torch

import torch.autograd as autograd

import torch.nn.functional as F





def train(train_iter, dev_iter, model, args):

#    if args.cuda:

 #       model.cuda()



    optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)



    steps = 0

    best_acc = 0

    last_step = 0

    model.train()

    for epoch in range(1, args.epochs+1):

        for batch in train_iter:

            feature, target = batch.text, batch.label

            feature.data.t_(), target.data.sub_(1)  # batch first, index align

            

            feature, target = feature.cuda(), target.cuda()



            optimizer.zero_grad()

            logit = model(feature)



            #print('logit vector', logit.size())

            #print('target vector', target.size())

            loss = F.cross_entropy(logit, target)

            loss.backward()

            optimizer.step()



            steps += 1

            if steps % args.log_interval == 0:

                corrects = (torch.max(logit, 1)[1].view(target.size()).data == target.data).sum()

                accuracy = 100.0 * corrects/batch.batch_size

                sys.stdout.write(

                    '\rBatch[{}] - loss: {:.6f}  acc: {:.4f}%({}/{})'.format(steps, 

                                                                             loss.data[0], 

                                                                             accuracy,

                                                                             corrects,

                                                                             batch.batch_size))

            if steps % args.test_interval == 0:

                dev_acc = eval(dev_iter, model, args)

                if dev_acc > best_acc:

                    best_acc = dev_acc

                    last_step = steps

                    if args.save_best:

                        save(model, args.save_dir, 'best', steps)

                else:

                    if steps - last_step >= args.early_stop:

                        print('early stop by {} steps.'.format(args.early_stop))

            elif steps % args.save_interval == 0:

                save(model, args.save_dir, 'snapshot', steps)





def eval(data_iter, model, args):

    model.eval()

    corrects, avg_loss = 0, 0

    for batch in data_iter:

        feature, target = batch.text, batch.label

        feature.data.t_(), target.data.sub_(1)  # batch first, index align

        feature, target = feature.cuda(), target.cuda()



        logit = model(feature)

        loss = F.cross_entropy(logit, target, size_average=False)



        avg_loss += loss.data[0]

        corrects += (torch.max(logit, 1)

                     [1].view(target.size()).data == target.data).sum()



    size = len(data_iter.dataset)

    avg_loss /= size

    accuracy = 100.0 * corrects/size

    print('\nEvaluation - loss: {:.6f}  acc: {:.4f}%({}/{}) \n'.format(avg_loss, 

                                                                       accuracy, 

                                                                       corrects, 

                                                                       size))

    return accuracy





def predict(text, model, text_field, label_feild, cuda_flag):

    assert isinstance(text, str)

    model.eval()

    # text = text_field.tokenize(text)

    text = text_field.preprocess(text)

    text = [[text_field.vocab.stoi[x] for x in text]]

    x = text_field.tensor_type(text)

    x = autograd.Variable(x, volatile=True)

    if cuda_flag:

        x = x.cuda()

    print(x)

    output = model(x)

    _, predicted = torch.max(output, 1)

    return label_feild.vocab.itos[predicted.data[0][0]+1]





def save(model, save_dir, save_prefix, steps):

    if not os.path.isdir(save_dir):

        os.makedirs(save_dir)

    save_prefix = os.path.join(save_dir, save_prefix)

    save_path = '{}_steps_{}.pt'.format(save_prefix, steps)

    torch.save(model.state_dict(), save_path)

In [0]:
import os

import argparse

import datetime

import torch

import torchtext.data as data

import torchtext.datasets as datasets

#import model

#import train

#import mydatasets


class args():
  lr=0.001
  epochs=200
  batch_size=64
  log_interval=1
  test_interval=100
  save_interval=500
  save_dir='snapshot'
  early_stop=1000
  save_best=True
  shuffle=False
  dropout=0.6
  max_norm=3.0
  embed_dim=256
  kernel_num=150
  kernel_sizes='1,2,3,4,5'
  static=False
  device=1
  no_cuda=False
  snapshot=None
  predict=None
  test=False
  kmax=5
  gru_l=1
  g_out=100
  #embed_num = len(text_field.vocab)
  #class_num = len(label_field.vocab) - 1
  #cuda = (not args.no_cuda) and torch.cuda.is_available(); del args.no_cuda
  #kernel_sizes = [int(k) for k in args.kernel_sizes.split(',')]
  #save_dir = os.path.join(args.save_dir, datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S'))
  
  
  

#parser = argparse.ArgumentParser(description='CNN text classificer')

# learning

#parser.add_argument('-lr', type=float, default=0.001, help='initial learning rate [default: 0.001]')

#parser.add_argument('-epochs', type=int, default=256, help='number of epochs for train [default: 256]')

#parser.add_argument('-batch-size', type=int, default=64, help='batch size for training [default: 64]')

#parser.add_argument('-log-interval',  type=int, default=1,   help='how many steps to wait before logging training status [default: 1]')

#parser.add_argument('-test-interval', type=int, default=100, help='how many steps to wait before testing [default: 100]')

#parser.add_argument('-save-interval', type=int, default=500, help='how many steps to wait before saving [default:500]')

#parser.add_argument('-save-dir', type=str, default='snapshot', help='where to save the snapshot')

#parser.add_argument('-early-stop', type=int, default=1000, help='iteration numbers to stop without performance increasing')

#parser.add_argument('-save-best', type=bool, default=True, help='whether to save when get best performance')

# data 

#parser.add_argument('-shuffle', action='store_true', default=False, help='shuffle the data every epoch')

# model

#parser.add_argument('-dropout', type=float, default=0.5, help='the probability for dropout [default: 0.5]')

#parser.add_argument('-max-norm', type=float, default=3.0, help='l2 constraint of parameters [default: 3.0]')

#parser.add_argument('-embed-dim', type=int, default=128, help='number of embedding dimension [default: 128]')

#parser.add_argument('-kernel-num', type=int, default=100, help='number of each kind of kernel')

#parser.add_argument('-kernel-sizes', type=str, default='3,4,5', help='comma-separated kernel size to use for convolution')

#parser.add_argument('-static', action='store_true', default=False, help='fix the embedding')

# device

#parser.add_argument('-device', type=int, default=-1, help='device to use for iterate data, -1 mean cpu [default: -1]')

#parser.add_argument('-no-cuda', action='store_true', default=False, help='disable the gpu')

# option

#parser.add_argument('-snapshot', type=str, default=None, help='filename of model snapshot [default: None]')

#parser.add_argument('-predict', type=str, default=None, help='predict the sentence given')

#parser.add_argument('-test', action='store_true', default=False, help='train or test')

#args = parser.parse_args()





# load SST dataset

#def sst(text_field, label_field,  **kargs):

 #   train_data, dev_data, test_data = datasets.SST.splits(text_field, label_field, fine_grained=True)

  #  text_field.build_vocab(train_data, dev_data, test_data)

   # label_field.build_vocab(train_data, dev_data, test_data)

    #train_iter, dev_iter, test_iter = data.BucketIterator.splits(
#
#                                       (train_data, dev_data, test_data), 
#
#                                       batch_sizes=(args.batch_size, 
#
#                                                     len(dev_data), 
#
#                                                     len(test_data)),

#                                        **kargs)

#    return train_iter, dev_iter, test_iter 





# load MR dataset

def mr(text_field, label_field, **kargs):

    train_data, dev_data = MR.splits(text_field, label_field)

    text_field.build_vocab(train_data, dev_data)

    label_field.build_vocab(train_data, dev_data)

    train_iter, dev_iter = data.Iterator.splits(

                                (train_data, dev_data), 

                                batch_sizes=(args.batch_size, len(dev_data)),

                                **kargs)

    return train_iter, dev_iter





# load data

print("\nLoading data...")

text_field = data.Field(lower=True)

label_field = data.Field(sequential=False)

train_iter, dev_iter = mr(text_field, label_field, device=-1, repeat=False)

# train_iter, dev_iter, test_iter = sst(text_field, label_field, device=-1, repeat=False)





# update args and print

args.embed_num = len(text_field.vocab)

args.class_num = len(label_field.vocab) - 1

#args.cuda = (not args.no_cuda) and torch.cuda.is_available(); del args.no_cuda

args.kernel_sizes = [int(k) for k in args.kernel_sizes.split(',')]

args.save_dir = os.path.join(args.save_dir, datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S'))



print("\nParameters:")

for attr, value in sorted(args.__dict__.items()):

    print("\t{}={}".format(attr.upper(), value))





# model

cnn = CNN_Text(args).cuda()

if args.snapshot is not None:

    print('\nLoading model from {}...'.format(args.snapshot))

    cnn.load_state_dict(torch.load(args.snapshot))



#if args.cuda:

#torch.cuda.set_device()

#cnn = cnn.cuda()
        



# train or predict

if args.predict is not None:

    label = train.predict(args.predict, cnn, text_field, label_field, args.cuda)

    print('\n[Text]  {}\n[Label] {}\n'.format(args.predict, label))

elif args.test:

    try:

        eval(test_iter, cnn, args) 

    except Exception as e:

        print("\nSorry. The test dataset doesn't  exist.\n")

else:

    print()

    try:

        train(train_iter, dev_iter, cnn, args)

    except KeyboardInterrupt:

        print('\n' + '-' * 89)

        print('Exiting from training early')

# Draft

In [0]:
for batch in train_iter:
    print(batch.label)


In [0]:
def kmax_pooling(x, dim, k):
    index = x.topk(k, dim = dim)[1].sort(dim = dim)[0]
    return x.gather(dim, index)

x = torch.rand(1, 7, 4, 5)



In [0]:
import torch

import torch.nn as nn

import torch.nn.functional as F

from torch.autograd import Variable

import numpy as np


class ll(nn.Module):
  
  def __init__(self):
    
    super(ll, self).__init__()
    
    self.cc=nn.ModuleList([nn.Conv2d(1, 100, (K, 4)) for K in [3,4,5]])
    
    self.dropout = nn.Dropout(0.5)

    self.fc1 = nn.Linear(len([3,4,5])*100*4, 2)
    
  def kmax_pooling(x, dim, k):
    
    index = x.topk(k, dim = dim)[1].sort(dim = dim)[0]
    
    return x.gather(dim, index)
    
  def forward(self,x):
    
    x = [conv(x).squeeze(3) for conv in self.cc]
    
    x = [kmax_pooling(i, 2, 4).squeeze(2) for i in x]
    
    x = torch.cat(x,1)
    
    x=x.view(x.size(0),-1)
    
    #x = self.dropout(x)  # (N, len(Ks)*Co)

    #x = self.fc1(x)  # (N, C)

    return x

In [0]:
import torch

import torch.nn as nn

import torch.nn.functional as F

from torch.autograd import Variable


class ll(nn.Module):
  
  def __init__(self):
    
    super(ll, self).__init__()
    
    self.cc=nn.ModuleList([nn.Conv2d(1, 100, (K, 4)) for K in [3,4,5]])
    
    self.dropout = nn.Dropout(0.5)

    self.fc1 = nn.Linear(len([3,4,5])*200, 2)
    
    self.rnn=nn.GRU(100,200,1)
    
  def kmax_pooling(x, dim, k):
    
    index = x.topk(k, dim = dim)[1].sort(dim = dim)[0]
    
    return x.gather(dim, index)
    
    
  def forward(self,x):
    
    x = [conv(x).squeeze(3) for conv in self.cc]
    
    x = [torch.transpose(i, 2, 0) for i in x]
    
    x = [torch.transpose(i, 2, 1) for i in x]
    
    x = [self.rnn(i)[1].squeeze(1) for i in x]
    
    
    
    x = torch.cat(x,1)
    
    x=x.view(x.size(0),-1)
    
    x = self.dropout(x)  # (N, len(Ks)*Co)

    x = self.fc1(x)  # (N, C)
    
    return x

In [322]:
x=Variable(torch.rand(1,40,200)).cuda()

#p=ll()

#u=p(x)

p=CNN_Text(args).cuda()

p(x)

#u


#u = [torch.transpose(i, 2, 0) for i in u]

#u = [torch.transpose(i, 2, 1) for i in u]

#rnn = nn.GRU(100, 100, 1)

#u = [rnn(i)[1] for i in u]



#u=torch.transpose(u, 2, 0)

#u=torch.transpose(u, 2, 1)



#rnn = nn.LSTM(100, 100, 1)

#output, hn = rnn(u)


Variable containing:
 0.0000 -0.1180
[torch.cuda.FloatTensor of size 1x2 (GPU 0)]

In [69]:
z = Variable(torch.rand(2, 3, 5))
z.t()

Variable containing:
(0 ,.,.) = 
  0.9864  0.1194  0.9939  0.9386  0.7324
  0.4424  0.9293  0.4787  0.2224  0.9564

(1 ,.,.) = 
  0.3188  0.0591  0.8272  0.2634  0.6105
  0.8466  0.3520  0.1231  0.1331  0.2259

(2 ,.,.) = 
  0.2348  0.7077  0.1340  0.1809  0.3188
  0.9936  0.5889  0.2112  0.6763  0.0063
[torch.FloatTensor of size 3x2x5]

In [45]:
x=Variable(torch.rand(1,1,40,4))

p=ll()

s=p(x)[0]

def kmax_pooling(x, dim, k):
    
    index = x.topk(k, dim = dim)[1].sort(dim = dim)[0]
    
    return x.gather(dim, index)

#s.topk(4, dim = 2)[1].sort(dim = 2)[0]

#kmax_pooling(s,2,4)

[kmax_pooling(i, 2, 4).squeeze(2) for i in p(x)]

RuntimeError: ignored

In [220]:
rnn = nn.GRU(100, 100, 2)
input = torch.randn(100, 1 ,100)
rnn(Variable(input))[1].squeeze(1)


Variable containing:

Columns 0 to 9 
-0.4315 -0.0872  0.3948  0.3150  0.0041  0.0375  0.5397  0.2245  0.4934  0.3367
 0.0040 -0.1512 -0.1987 -0.1027 -0.0789 -0.0378  0.0512  0.0830 -0.1289 -0.1766

Columns 10 to 19 
-0.0272 -0.2784 -0.4954  0.2914 -0.4315  0.3496  0.3775  0.2233  0.1320  0.0389
 0.1935 -0.1346  0.0160 -0.1850  0.0617 -0.1704  0.0322  0.0400 -0.4134  0.0483

Columns 20 to 29 
 0.2431 -0.0432 -0.3716 -0.3888 -0.4464  0.1502  0.0768  0.2435  0.4199 -0.4847
-0.2294  0.0129 -0.0168 -0.3497  0.0461 -0.2200  0.0180 -0.0607 -0.0598 -0.2125

Columns 30 to 39 
 0.0708  0.0165  0.5297 -0.0770  0.4444  0.1545 -0.3218 -0.0366 -0.0221  0.3145
-0.2074  0.1367 -0.2058  0.0369  0.1248  0.0607  0.2150  0.0263  0.0605 -0.3468

Columns 40 to 49 
-0.2220  0.0920 -0.6153 -0.0299  0.1329 -0.1236  0.1708 -0.5577  0.0240  0.0146
 0.1071 -0.0852  0.1101  0.0991 -0.0776  0.0403  0.0118  0.2446  0.2504  0.1969

Columns 50 to 59 
-0.4161 -0.4925  0.1867  0.5797 -0.3331  0.0798  0.0577  0.2673 -0.